In [1]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


### Непонятные признаки

- fnlwgt - final weight, сколько человек представляет данная выборка
- educational-num - численное обозначение уровня образования (education)

In [5]:
data.shape

(48842, 15)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


#### Хорошие новости! В датасете отсутствуют null значения.

Построим модель используя признаки workclass (рабочий класс) и education (образование)

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = data[['workclass', 'educational-num']]

# столбец sport является категориальной переменной
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies(selectedColumns, columns = ['workclass'])

X.head()

,educational-num,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,7,0,0,0,0,1,0,0,0,0
1,9,0,0,0,0,1,0,0,0,0
2,12,0,0,1,0,0,0,0,0,0
3,10,0,0,0,0,1,0,0,0,0
4,10,1,0,0,0,0,0,0,0,0


In [33]:
# целевая переменная (столбец income) снова является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(data['income'])

# записываем в переменную y преобразованный столбец income
y = pd.Series(data = le.transform(data['income']))

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
model = LogisticRegression(C = 10) 

In [36]:
# обучаем модель

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

C:\Users\Ivan\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [37]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.97      0.87      7479
           1       0.62      0.17      0.26      2290

    accuracy                           0.78      9769
   macro avg       0.71      0.57      0.57      9769
weighted avg       0.75      0.78      0.73      9769



In [38]:
from sklearn.metrics import r2_score

print(r2_score(y_test, y_pred))

-0.22006193761746884


In [39]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))

0.7810420718599652


Построем другую модель, используя признаки age, race и gender

In [40]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = data[['age', 'race', 'gender']]

# столбец sport является категориальной переменной
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies(selectedColumns, columns = ['race', 'gender'])

X.head()

,age,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,gender_Female,gender_Male
0,25,0,0,1,0,0,0,1
1,38,0,0,0,0,1,0,1
2,28,0,0,0,0,1,0,1
3,44,0,0,1,0,0,0,1
4,18,0,0,0,0,1,1,0


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
model = LogisticRegression(C = 1) 

In [43]:
# обучаем модель

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

C:\Users\Ivan\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [44]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.96      0.85      7479
           1       0.28      0.05      0.09      2290

    accuracy                           0.75      9769
   macro avg       0.53      0.51      0.47      9769
weighted avg       0.65      0.75      0.67      9769



In [45]:
print(r2_score(y_test, y_pred))

-0.41057184279008885


In [46]:
print(accuracy_score(y_test, y_pred))

0.7468522878493192


Построим ещё одну модель используя признаки age и occupation

In [47]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = data[['age', 'occupation']]

# столбец sport является категориальной переменной
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies(selectedColumns, columns = ['occupation'])

X.head()

,age,occupation_?,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,25,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,38,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,28,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,44,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,18,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
model = LogisticRegression(C = 1) 

In [50]:
# обучаем модель

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

C:\Users\Ivan\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [51]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.94      0.86      7479
           1       0.53      0.23      0.32      2290

    accuracy                           0.77      9769
   macro avg       0.67      0.59      0.59      9769
weighted avg       0.74      0.77      0.74      9769



In [52]:
print(r2_score(y_test, y_pred))

-0.2702561641300154


In [53]:
print(accuracy_score(y_test, y_pred))

0.7720339850547651
